In [151]:
import pandas as pd
import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from Bio import Entrez
from scipy.cluster.hierarchy import dendrogram, linkage
import subprocess
import multiprocessing
import plotly.express as px

In [152]:
coverage_file = 'DATA/external_rna_not_paired/calculated_coverage_and_quality/ERR3491123_coverage.txt'
tables_folder = 'ictv_tables/'

In [153]:
ictv_data = pd.read_excel(f'{tables_folder}/ictv_taxo.xlsx')
taxo_index = pd.read_csv(f'{tables_folder}/genbank_accessions.csv', index_col=0)

In [182]:
def get_data_table(coverage_file):
    data = pd.read_table(coverage_file).rename(columns={'#rname': 'rname'}).drop(['numreads', 'covbases',
           'meandepth', 'meanbaseq', 'meanmapq'], axis=1)

    data['rname_short'] = data['rname'].apply(lambda x: x.split('|')[1])
    data = data.set_index('rname_short')

    data = data.copy()
    data = pd.merge(left=data.reset_index(), 
                    right=taxo_index.reset_index(), left_on='rname_short', right_on='index') # Get index of all accession number
    data = pd.merge(left=data, right=ictv_data.reset_index(), 
                    left_on='ictv_taxo_index', right_on='index') # Get ictv metadata by accession number


    # Generalize fragments of one virus
    data[['coverage']] = data[['coverage']]  * 0.01
    data['len'] = data.endpos - data.startpos
    data[['weighted_coverage']] = (data[['coverage']].T * data['len']).T


    agg_all_columns_dict = dict()
    agg_all_columns_dict.update({'len':'sum', 'weighted_coverage':'sum'})
    agg_all_columns_dict.update({'coverage': lambda x: [i for i in list(x)]})
    agg_all_columns_dict.update({'endpos': lambda x: [i for i in list(x)]})
    agg_all_columns_dict.update({'rname': lambda x: [i for i in list(x)]})
    agg_all_columns_dict.update({i:'first' for i in ['Realm', 'Kingdom',
           'Subkingdom', 'Phylum', 'Subphylum', 'Class', 'Order', 'Family', 'Genus', 'Species',
                  'Virus name(s)', 'Virus GENBANK accession', 'Host source']})
    data = data.groupby('Species').agg(agg_all_columns_dict)
    data['fragments_coverage'] = data['coverage'].copy()


    data['coverage'] = data.weighted_coverage / data.len
    data = data.sort_values(by='coverage', ascending=False)
    data = data[['endpos', 'rname', 'coverage', 'Species']]
    data = data.query('coverage > 0.2')
    return data

In [188]:
files = glob.glob('DATA/external_rna_not_paired/calculated_coverage_and_quality/*_coverage.txt')

In [192]:
for coverage_file in files:
    data = get_data_table(coverage_file)
    name = os.path.splitext(os.path.basename(coverage_file))[0].replace('_coverage', '')
    bam_folder = 'DATA/external_rna_not_paired/unclassified_sorted_bam/'

    for endposes, rnames, spec in zip(data.endpos, data.rname, data.Species):
        for endpos, rname in zip (endposes, rnames):
            cmd = f'''bamsnap -draw coordinates bamplot coverage base 
            -out DATA/external_rna_not_paired/drawings/{spec.replace(' ', '_')}/{rname.replace('|', '_')}_{name}.png 
            -bam {bam_folder}/{name}.sorted.bam 
            -pos '{rname}:600-{endpos}' 
            -ref DATA/bats/all_virus_reference/all_genomes.fasta 
            -bamplot coverage read 
            -draw coordinates bamplot -no_target_line'''.replace('\n', '')
            print('..')
            !{cmd}

..
WW
2023-05-22 14:10:50,916 : [INFO] (proc 1) Saved DATA/external_rna_not_paired/drawings/Alphainfluenzavirus_influenzae/ENA_J02146_J02146.1_ERR3491130.png
2023-05-22 14:10:50,916 : [INFO] (proc 1) Saved DATA/external_rna_not_paired/drawings/Alphainfluenzavirus_influenzae/ENA_J02146_J02146.1_ERR3491130.png : 0.05647 sec
2023-05-22 14:10:50,920 : [INFO] Total running time: 0.1 sec
..
WW
2023-05-22 14:10:51,361 : [INFO] (proc 1) Saved DATA/external_rna_not_paired/drawings/Alphainfluenzavirus_influenzae/ENA_J02147_J02147.1_ERR3491130.png
2023-05-22 14:10:51,361 : [INFO] (proc 1) Saved DATA/external_rna_not_paired/drawings/Alphainfluenzavirus_influenzae/ENA_J02147_J02147.1_ERR3491130.png : 0.07909 sec
2023-05-22 14:10:51,366 : [INFO] Total running time: 0.1 sec
..
WW
2023-05-22 14:10:51,796 : [INFO] (proc 1) Saved DATA/external_rna_not_paired/drawings/Alphainfluenzavirus_influenzae/ENA_J02150_J02150.1_ERR3491130.png
2023-05-22 14:10:51,796 : [INFO] (proc 1) Saved DATA/external_rna_not_pa

KeyboardInterrupt: 